In [61]:
import os
import yaml
from datetime import datetime

import pandas as pd
import psycopg2
from sshtunnel import SSHTunnelForwarder

In [7]:
def make_output_path():
    return os.path.join(
        'exports',
        "{}-{}".format(
            datetime.now().strftime("%Y-%m-%dT%H"),
            os.path.basename(EXCEL_FILE_PATH).replace('xlsx', 'csv')
        )
    )

In [8]:
with open('private/secrets.yml') as in_file:
    secrets = yaml.load(in_file)

In [9]:
def query(q):
    with SSHTunnelForwarder(
        ('hkgmetadb.infra.ooni.io', 22),
        ssh_username='art',
        ssh_private_key=secrets['ssh_private_key_path'],
        remote_bind_address=('localhost', 5432)
    ) as server:
        conn = psycopg2.connect(
            host='localhost',
            port=server.local_bind_port,
            user='shovel',
            password=secrets['shovel_password'],
            dbname='metadb')
        return pd.read_sql_query(q, conn)

In [47]:
EXCEL_FILE_PATH = "Data Request UA_2.xlsx"
xls = pd.read_excel(EXCEL_FILE_PATH).reset_index()

In [48]:
xls.head()

,index,Countries,Start date,End Date
0,url,UA,beginning of times,2018-04-15
1,http://24tv.ua,NaN,NaN,NaT
2,http://2ch.hk,NaN,NaN,NaT
3,http://3a-pycb.livejournal.com,NaN,NaN,NaT
4,http://6ott-mit-uns.livejournal.com,NaN,NaN,NaT


In [49]:
countries = list(map(lambda x: x.strip(), xls.iloc[0]['Countries'].split(',')))
start_date = xls.iloc[0]['Start date']
end_date = xls.iloc[0]['End Date']
urls = list(xls[xls['index'].notnull()]['index'])

In [50]:
urls.pop(0)

'url'

In [51]:
start_date

'beginning of times'

In [52]:
end_date

Timestamp('2018-04-15 00:00:00')

In [53]:
base_query = """SELECT measurement.test_runtime,
input.input,
measurement.measurement_start_time,
report.probe_cc,
report.probe_asn,
report.probe_ip,
report.report_id,
http_verdict.http_experiment_failure,
http_verdict.blocking
FROM measurement 
JOIN input ON input.input_no = measurement.input_no 
JOIN report ON report.report_no = measurement.report_no
JOIN http_verdict ON http_verdict.msm_no = measurement.msm_no
"""
where_clause = "WHERE ({}) AND ({})".format(
    " OR ".join(list(map(lambda x: "report.probe_cc = '{}'".format(x), countries))),
    " OR ".join(list(map(lambda x: "input = '{}'".format(x), urls))),
)

In [54]:
rows = query(base_query + where_clause)

In [18]:
rows[
    (rows['measurement_start_time'] < end_date)
].to_csv(make_output_path())

In [56]:
rows.to_csv(make_output_path())